# Problem Set 04: HMMs

In this problem set you will implement value iteration in a non-deterministic MDP, and HMM Filtering.


0. [Credit for Contributors (required)](#contributors)

      
1. [HMM Filtering](#problem1)
    1. [Implement `compute_casino_table` for fair dice estimation (40 points)](#compute_casino_table)
    2. [Dendrochronology Example (20 points)](#viterbi)
    
**60 points** total for Problem Set 4



## <a name="contributors"></a> Credit for Contributors

List the various students, lecture notes, or online resouces that helped you complete this problem set:

Ex: I worked with Bob on the cat activity planning problem.

<div class="alert alert-info">
Write your answer in the cell below this one.
</div>

Hidden Markov Models Lecture Notes
AIMA Ch 14.2


In [1]:
# Useful imports

from __future__ import division

%matplotlib inline
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import copy
from nose.tools import ok_, assert_equal, assert_almost_equal

## <a name="problem1"/></a> Problem 1 : HMM  Filtering (60 points)

In this problem you will perform filtering on an HMM using the forward algorithm (as explained in class).

Remember that the filtering problem answers the question

\begin{equation}
Pr(S_t | o_{1:t})
\end{equation}

that is, what is the most likely state at time $t$ after reading $t$ observations.

You will implement filtering and demonstrate it for the "dishonest casino" example HMM that was introduced in class. The parameters for the HMM are shown in the following image (and also in the lecture notes).

<img src='hmm_casino.png'/>

Also, the prior for the states is

\begin{align}
    &Pr(S_0 = Fair) &= 0.8\\
    &Pr(S_0 = Loaded) &= 0.2
\end{align}

that is, the probability that the initial state is "Fair" is 0.8, and the probability that the initial state is "Loaded" is 0.2.

Implement filtering for this HMM for the following observation sequence:

`['1','2','4','6','6','6','3','6']`

Your code should produce a table like the following one (from 0 measurements to the totality of the measurements provided).


```
Filtering for the dishonest casino HMM:
  with 0 measurement(s): [], P(s at t=0) is: [Fair: 0.8000,  Loaded: 0.2000]
  with 1 measurement(s): ['1'], P(s at t=1) is: [Fair: 0.8480,  Loaded: 0.1520]
  with 2 measurement(s): ['1', '2'], P(s at t=2) is: [Fair: 0.8789,  Loaded: 0.1211]
  with 3 measurement(s): ['1', '2', '4'], P(s at t=3) is: [Fair: 0.8981,  Loaded: 0.1019]
  with 4 measurement(s): ['1', '2', '4', '6'], P(s at t=4) is: [Fair: 0.6688,  Loaded: 0.3312]
  ...
```

Note that this is exactly the example given in class and that you have the results in the slides.

<div class="alert alert-info">
Implement HMM filtering and use it in the "dishonest casino" example HMM to generate a table like the one above.
</div>


**Note**: you could implement `smoothing` and `decoding` on the same HMM model and observation data for **additional credit** (but you don't have to if you don't want). 

If you do implement smoothing and decoding, please make it explicit in a separate cell below for our graders to see.

In [6]:
# You might find the following examples on how to use numpy for matrices and vectors useful

# Define a 2x3 Matrix
M=np.array([[3.5, 2.1, 1.3],
            [0.1, 0.2, 0.3]])
print("Matrix M is:")
print(M)

print("Element (0,1) of M is: ", M[0,1])
print("Shape (dimension) of M is: ", M.shape)

# Define 1x2 vector
v =  np.array([1, 3])
print("Vector v is:")
print(v)

# Matrix multiplication r = v*M
r = v.dot(M)
print("Matrix multiplication v * M = ", r)

# Element wise multiplication
a = np.array([2, 4])
print("Element wise v * a = %s * %s = "%(v,a), v*a)

# Sum of elements of v
print("Sum of elements of v (%s) is: %s" %(v, np.sum(v)))

# Vector slicing
b = np.array([2,4,6,8,10,12,14,16])
print("Vector b is: ", b)
print("First 3 elements of b are: ", b[:3])
print("Elements from the second until the end are: ", b[1:])
print("Elements from 4th to 7th are: ", b[3:7])

Matrix M is:
[[3.5 2.1 1.3]
 [0.1 0.2 0.3]]
Element (0,1) of M is:  2.1
Shape (dimension) of M is:  (2, 3)
Vector v is:
[1 3]
2
Matrix multiplication v * M =  [3.8 2.7 2.2]
Element wise v * a = [1 3] * [2 4] =  [ 2 12]
Sum of elements of v ([1 3]) is: 4
Vector b is:  [ 2  4  6  8 10 12 14 16]
First 3 elements of b are:  [2 4 6]
Elements from the second until the end are:  [ 4  6  8 10 12 14 16]
Elements from 4th to 7th are:  [ 8 10 12 14]


 ###  <a name="compute_casino_table"></a>Implement HMM Filter

Write your code as a function:

```
def compute_casino_table(meas)
```

that takes as input the observation sequence, and prints the table discussed above.


In [57]:
def compute_casino_table(meas):
    # Example observation sequence
    # meas = ['1','2','4','6','6','6','3','6']
    
    # YOUR CODE HERE
    t = int(len(meas))
    table = np.zeros([t+1,2])
    for i in range(t+1):
        if i == 0:
            table[i,0] = 0.8 #Pr(S0=Fair)=0.8
            table[i,1] = 0.2 #Pr(S0=Fair)=0.2
        else:
            rolled = meas[i-1]
            if rolled == '6':
                p_fair = (1/6)*((0.95*table[i-1,0])+(0.05*table[i-1,1]))
                p_loaded = (1/2)*((0.05*table[i-1,0])+(0.95*table[i-1,1]))
                alpha = 1/(p_fair+p_loaded)
                table[i,0] = alpha*p_fair
                table[i,1] = alpha*p_loaded
            else:
                p_fair = (1/6)*((0.95*table[i-1,0])+(0.05*table[i-1,1]))
                p_loaded = (1/10)*((0.05*table[i-1,0])+(0.95*table[i-1,1]))
                alpha = 1/(p_fair+p_loaded)
                table[i,0] = alpha*p_fair
                table[i,1] = alpha*p_loaded
    print("Filtering for the dishonest casino HMM:")
    for i in range(t+1):
        print(f"\t with {i} measurement(s): {meas[:i]}, P(s at t={i}) is: [Fair: {round(table[i,0],4)}, Loaded: {round(table[i,1],4)}]")

In [58]:
# Don't modify this cell
meas = ['1','2','4','6','6','6','3','6']
compute_casino_table(meas)

Filtering for the dishonest casino HMM:
	 with 0 measurement(s): [], P(s at t=0) is: [Fair: 0.8, Loaded: 0.2]
	 with 1 measurement(s): ['1'], P(s at t=1) is: [Fair: 0.848, Loaded: 0.152]
	 with 2 measurement(s): ['1', '2'], P(s at t=2) is: [Fair: 0.8789, Loaded: 0.1211]
	 with 3 measurement(s): ['1', '2', '4'], P(s at t=3) is: [Fair: 0.8981, Loaded: 0.1019]
	 with 4 measurement(s): ['1', '2', '4', '6'], P(s at t=4) is: [Fair: 0.6688, Loaded: 0.3312]
	 with 5 measurement(s): ['1', '2', '4', '6', '6'], P(s at t=5) is: [Fair: 0.3843, Loaded: 0.6157]
	 with 6 measurement(s): ['1', '2', '4', '6', '6', '6'], P(s at t=6) is: [Fair: 0.1793, Loaded: 0.8207]
	 with 7 measurement(s): ['1', '2', '4', '6', '6', '6', '3'], P(s at t=7) is: [Fair: 0.3088, Loaded: 0.6912]
	 with 8 measurement(s): ['1', '2', '4', '6', '6', '6', '3', '6'], P(s at t=8) is: [Fair: 0.1399, Loaded: 0.8601]


 ###  <a name="viterbi"></a>Dendrochronology Example
 
Dendrochronology (or tree-ring dating) is the scientific method of dating tree rings (also called growth rings) to the exact year they were formed in order to analyze atmospheric conditions during different periods in history. [Wikipedia]

<img src="tree-rings-diagram.jpg" style="width:40%;"/>
<p style="text-align:center">Image source: NASA </p>

Let’s say we are interested in estimating the average annual temperature at a particular location
on Earth. However, the time period that we are interested in is in distant past, with no direct
temperature data available. Prior research indicates correlation between size of tree rings and temperature, so we decide to use this as our evidence.

To simplify the problem, we’ll only consider two temperature states, “Hot” and “Cold”. Suppose
that we are told the probability of hot year followed by another hot year is 0.7 and the probability
that a cold year is followed by another cold year is 0.6. Assume that the initial state distribution, π, is given. For observations, we’ll only consider three different tree ring sizes, small, medium, and large, or S, M , and L, respectively. The probabilistic relationship between the annual temperature and the ring sizes is the observation matrix. The state transition matrix, initial distribution, and observation matrix are provided below:

<img src="temp_example.png"/>

For example, the probability of observing a “Large” tree ring size given that the state is “Hot” is 0.5.


From a particular four-year period of interest, we observe the series of tree rings, **[S, M, S, L]**. 
Assume that $\pi$ is for $s_0$ and the observations are for $s_1, \ldots, s_4$ i.e $o_1 = $ **S** and so on. 

With this observation sequence, find the most likely temperature sequence $[s_1, s_2, s_3, s_4]$ using the **Viterbi
algorithm**. 

Please show how you found the most likely sequence.


<div class="alert alert-info">
Write or upload your answer in the cell below this one.
</div>

YOUR ANSWER HERE

In [7]:
dict_all = dict()
dict_all["H"] = {"H":0.7,"C":0.3,"S":0.1,"M":0.4,"L":0.5}
dict_all["C"] = {"H":0.4,"C":0.6,"S":0.7,"M":0.2,"L":0.1}
def viterbi(observations):
    t = int(len(observations))
    table = np.zeros([t+1,2])
    for i in range(t+1):
        if i == 0:
            table[i,0] = 0.6
            table[i,1] = 0.4
        else:
            observation = observations[i-1] # = X_t+1
            prev_hot = table[i-1,0]
            prev_cold = table[i-1,1]
            # for hot
            H_to_H = prev_hot*dict_all["H"]["H"]
            C_to_H = prev_cold*dict_all["C"]["H"]
            p_evid_hot = dict_all["H"][observation]
            table[i,0] = max(H_to_H,C_to_H)*p_evid_hot
            # for cold
            H_to_C = prev_hot*dict_all["H"]["C"]
            C_to_C = prev_cold*dict_all["C"]["C"]
            p_evid_cold = dict_all["C"][observation]
            table[i,1] = max(H_to_C,C_to_C)*p_evid_cold
    most_possible = []
    for i in range(1,t+1):
        if table[i,0]>table[i,1]:
            most_possible.append("H")
        else:
            most_possible.append("C")
    print(most_possible)
    return most_possible
observations = ["S","M","S","L"]
viterbi(observations)

['C', 'H', 'C', 'H']


['C', 'H', 'C', 'H']